In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import fiona
import geopandas as gpd
from scipy import spatial
%matplotlib inline

In [85]:
pd.options.display.max_rows = 10

## Read SWMM shapefiles as geopandas dataframes

In [48]:
junctions = gpd.read_file('/Users/mdbartos/Data/GIS/Ann Arbor/full ann arbor 090216/junctions.shp')
conduits = gpd.read_file('/Users/mdbartos/Data/GIS/Ann Arbor/full ann arbor 090216/conduits.shp')
orifices = gpd.read_file('/Users/mdbartos/Data/GIS/Ann Arbor/full ann arbor 090216/orifices.shp')
outfalls = gpd.read_file('/Users/mdbartos/Data/GIS/Ann Arbor/full ann arbor 090216/outfalls.shp')
outlets = gpd.read_file('/Users/mdbartos/Data/GIS/Ann Arbor/full ann arbor 090216/outlets.shp')
pumps = gpd.read_file('/Users/mdbartos/Data/GIS/Ann Arbor/full ann arbor 090216/pumps.shp')
storages = gpd.read_file('/Users/mdbartos/Data/GIS/Ann Arbor/full ann arbor 090216/storages.shp')
weirs = gpd.read_file('/Users/mdbartos/Data/GIS/Ann Arbor/full ann arbor 090216/weirs.shp')

## Combine all "junction-like" objects into a single table

In [49]:
vertices = pd.concat([junctions, storages, outfalls], join='inner').reset_index(drop=True)

## Combine all "link-like" objects into a single table

In [50]:
links = pd.concat([conduits, orifices, pumps, outlets, weirs], join='inner').reset_index(drop=True)

## Extract the xy coordinates of the junctions

In [51]:
vertices_xy = np.vstack(vertices.geometry.apply(lambda x: np.concatenate(x.coords.xy)).values)
vertices_xy

array([[ 13298391.508,    291805.924],
       [ 13285267.384,    295525.859],
       [ 13286184.797,    296973.833],
       ..., 
       [ 13296802.986,    287114.311],
       [ 13296710.542,    286709.607],
       [ 13302477.603,    302519.355]])

## Extract the xy coordinates of the start and end of each link

In [63]:
link_inlets = np.vstack(links.geometry.apply(lambda x: np.column_stack(x.coords.xy)[0, :]).values)
link_outlets = np.vstack(links.geometry.apply(lambda x: np.column_stack(x.coords.xy)[-1, :]).values)
link_inlets, link_outlets

(array([[ 13285267.384,    295525.859],
        [ 13286184.797,    296973.833],
        [ 13286258.426,    297011.181],
        ..., 
        [ 13303639.458,    271883.141],
        [ 13303502.242,    295524.371],
        [ 13298405.603,    296030.644]]),
 array([[ 13285467.258,    295720.392],
        [ 13286258.426,    297011.181],
        [ 13286314.052,    297045.973],
        ..., 
        [ 13303589.857,    271838.652],
        [ 13303195.531,    295268.687],
        [ 13298289.976,    295867.246]]))

## Construct a spatial index on the junction coordinates

In [54]:
tree = spatial.cKDTree(vertices_xy)

## Query the spatial index for the link start and end coordinates

In [55]:
inlet_dist, inlet_ix = tree.query(link_inlets)
outlet_dist, outlet_ix = tree.query(link_outlets)

## Check the max distance between the paired junction and link coordinates;

In [69]:
# Turns out the max distance is zero, which means they matched exactly
# (as they should in this case)

inlet_dist.max(), outlet_dist.max()

(0.0, 0.0)

## Use the computed indices from the query to get the junctions for each link

In [71]:
inlet_ix, outlet_ix

(array([    1,     2,     3, ..., 12292, 12223, 12218]),
 array([ 2541,     3,     4, ..., 11863,    89, 11758]))

In [86]:
# The junctions for each inlet are:
vertices['NAME'].iloc[inlet_ix]

1         36504_1
2         37026_1
3         37026_2
4         37026_3
5         37035_1
           ...   
12216    93-50415
12211    93-50407
12292     RITEAID
12223    93-50432
12218    93-50423
Name: NAME, dtype: object

In [87]:
# The junctions for each outlet are:
vertices['NAME'].iloc[outlet_ix]

2541       91-51831
3           37026_2
4           37026_3
1931       91-50042
1934       91-50047
            ...    
11741      98-50088
733        88-56287
11863    RITEAIDOUT
89         88-50353
11758      98-50108
Name: NAME, dtype: object

## Check that the computed junctions match the ones listed under 'INLETNODE' and 'OUTLETNODE'

In [79]:
(links['INLETNODE'].values == vertices['NAME'].iloc[inlet_ix].values).all()

True

In [80]:
(links['OUTLETNODE'].values == vertices['NAME'].iloc[outlet_ix].values).all()

True